In [3]:
from bs4 import BeautifulSoup
import requests
import json
import re
import sys
import pprint

In [39]:
uri_file = open('icolinks.json').read()
uri_list = json.loads(uri_file)
print('we have %d uris' % len(uri_list))

we have 1719 uris


In [77]:
regex = re.compile(r"\s+")
num_uris = 1719
ico_list = []
for i, uri in enumerate(uri_list):
    ico_info = {}
    base_url = 'https://www.icodata.io'
    soup = BeautifulSoup(requests.get(base_url + uri).text, 'html.parser')
    info = soup.find('div', 'other-info')
    vals = soup.find('p', 'dates')
    name = regex.sub("", soup.find('h1', 'coin-name').text)
    ico_info['name']= name
    for c in info.contents:
        if 'NavigableString' not in str(type(c)) and c.text.strip() is not "":
            text = c.text.strip()
            if 'Coinmarketcap' in text:
                ico_info['cmc_rank'] = text.split(':')[1]
            if 'Description' in text:
                ico_info['description'] = text.split(':')[1]
    for c in vals.text.split('\n'):
        if c.strip() is not "":
            if 'Start of ICO' in c:
                ico_info['ico_start'] = c.split('  ')[1]
            if 'End of ICO' in c:
                ico_info['ico_end'] = c.split('  ')[1]
            if 'Hard cap' in c:
                ico_info['hard_cap'] = c.split(' ')[2]
            if 'Raised' in c:
                ico_info['raised'] = c.split(' ')[1]
            if 'Token Sale Price' in c:
                ico_info['price'] = c.split(' ')[3]
    ico_list.append(ico_info)
    sys.stdout.write('\r' + ' '*100)
    sys.stdout.write('\r' + '{} out of {} : {}'.format(i, num_uris, name))
icoJSON = json.dumps(ico_list)
with open('ico_info_list.json', 'w') as f:
    f.write(icoJSON)

1718 out of 1719 : CannaSOS                                                                         

In [82]:
with open('ico_info_list.json', 'r') as f:
    data = json.loads(f.read())
    for i, d in enumerate(data):
        d['ico_start'] = d['ico_start'].strip()
        d['ico_end'] = d['ico_end'].strip()
        d['raised'] = float(d['raised'][1:].replace(",", ""))
        if 'cmc_rank' in d and d['cmc_rank'] != "":
            d['cmc_rank'] = int(d['cmc_rank'])
        if 'hard_cap' in d:
            d['hard_cap'] = float(d['hard_cap'][1:].replace(",", ""))
        if 'price' in d:
            d['price'] = float(d['price'][1:].replace(",", ""))
        if 'description' in d:
            d['description'] = d['description'].strip()
        sys.stdout.write('\r' + ' '*100)
        sys.stdout.write('\r' + '{} out of {} : {}'.format(i, num_uris, d['name']))
with open('ico_info_list_cleaned.json', 'w') as f:
    print('\n' + 'writing to file ...')
    f.write(json.dumps(data))
    f.close()

1718 out of 1719 : CannaSOSwriting to file ...
